In [29]:
import pandas as pd
import numpy as np
from functions import *
from pytorch_util import *
from pytorch_models import ConvBatchLeaky,GRU_NCL,LambdaLayer
from torch.nn import Linear,Sequential,LeakyReLU,Dropout,L1Loss,MaxPool2d,MaxPool1d
from torch.utils.data import DataLoader
from torch.optim import Adam

In [2]:
train = pd.read_csv("../Data/train.csv", 
                    dtype={'acoustic_data': np.float32, 'time_to_failure': np.float32})
train = train.values

In [3]:
## change ##
batch_size = 16
clip = .5

Prepare data

In [4]:
train_gen = SequenceGenSpec(train,Is2D=False)
train_gen = DataLoader(train_gen,batch_size,False,num_workers=2)

val_gen = SequenceGenSpec(train,False,Is2D=False)
val_gen = DataLoader(val_gen,batch_size,False,num_workers=2)

Build model

In [73]:
model = Sequential(Dropout(0.25),
                      ConvBatchLeaky(1,4,(5,3),stride=(2,1)),
                      Dropout(0.25),
                      ConvBatchLeaky(4,8,(5,3),stride=(2,1)),
                      Dropout(0.25),
                      ConvBatchLeaky(8,16,(5,5),stride=(2,2)),
                      Dropout(0.25),
                      ConvBatchLeaky(16,32,(3,3),stride=(2,2)),
                      #MaxPool2d((6,1)),
                      Dropout(0.25),
                      ConvBatchLeaky(32,32,(6,1),stride=(1,1)),
                      Dropout(0.25),
                      LambdaLayer(lambda x:x.squeeze(2)),
                      DenseBlock(32),
                      Dropout(0.5),                  
                      DenseBlock(64),
                      Dropout(0.5),
                      DenseBlock(128),
                      Dropout(0.5),                 
                      DenseBlock(256),
                      Dropout(0.5),             
                      DenseBlock(512),
                      MaxPool1d(5),
                      Dropout(0.5),                      
                      LambdaLayer(lambda x:x.squeeze(2)),
                      Linear(1024,128),LeakyReLU(0.1,True),
                      Dropout(0.25),Linear(128,1),LeakyReLU(0.1,True),
                      LambdaLayer(lambda x:x.squeeze(1))
                     ).to('cuda:0')

In [74]:
opt = Adam(trainable_parameter(model),lr=1e-4)

In [75]:
loss_func_nor = loss_func_generator(L1Loss())

Training

In [76]:
model = fit(5, model, loss_func_nor, opt, train_gen, val_gen)

epoch:0, train_loss:3.2217936985767803, val_loss:5.280186653137207
epoch:1, train_loss:3.1231767048056307, val_loss:5.048975467681885
epoch:2, train_loss:3.118224114179611, val_loss:4.890585422515869
epoch:3, train_loss:3.0939547442472897, val_loss:4.5919976234436035
epoch:4, train_loss:3.0478792752210913, val_loss:4.2235941886901855
Training completed in 29.753233432769775s


In [77]:
model = fit(5, model, loss_func_nor, opt, train_gen, val_gen)

epoch:0, train_loss:2.8925352457624216, val_loss:3.603809356689453
epoch:1, train_loss:2.3964541737849894, val_loss:3.5175061225891113
epoch:2, train_loss:2.327695711874045, val_loss:3.597337245941162
epoch:3, train_loss:2.286822204406445, val_loss:3.6284518241882324
epoch:4, train_loss:2.288866975559638, val_loss:3.633211851119995
Training completed in 29.998956441879272s


Submission: cnn does not work well